In [59]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.context import SparkContext, SparkConf
from pyspark.sql.types import StructType, StringType, IntegerType, ArrayType
import pandas as pd

In [2]:
spark = SparkSession.builder.appName('test').master('local[*]').getOrCreate()

In [3]:
sc = spark.sparkContext

In [4]:
df = spark.read.csv('stu_score.txt', sep=',', header=False)

In [5]:
df.show(2)

+---+----+---+
|_c0| _c1|_c2|
+---+----+---+
|  1|语文| 99|
|  2|语文| 99|
+---+----+---+
only showing top 2 rows



In [6]:
df2 = df.toDF('id', 'name', 'score')

In [7]:
df2.printSchema()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- score: string (nullable = true)



In [8]:
df2.show(3)

+---+----+-----+
| id|name|score|
+---+----+-----+
|  1|语文|   99|
|  2|语文|   99|
|  3|语文|   99|
+---+----+-----+
only showing top 3 rows



In [9]:
df2.createTempView('score')

# df2.createGlobalTempView

In [10]:
#SQL
spark.sql("""
    SELECT * FROM score WHERE name ='语文' LIMIT 5
""").show()
 


+---+----+-----+
| id|name|score|
+---+----+-----+
|  1|语文|   99|
|  2|语文|   99|
|  3|语文|   99|
|  4|语文|   99|
|  5|语文|   99|
+---+----+-----+



In [11]:
# DSL
df2.where("name = '语文'") . limit(5). show()

+---+----+-----+
| id|name|score|
+---+----+-----+
|  1|语文|   99|
|  2|语文|   99|
|  3|语文|   99|
|  4|语文|   99|
|  5|语文|   99|
+---+----+-----+



# TXT  rdd to DF

# 1

In [12]:
rdd = sc.textFile('people.txt')
rdd.collect()

['zhangsan, 23', 'sb, 49', 'wm, 90']

In [13]:
rdd = rdd.map(lambda x: x.split(','))
rdd.collect()

[['zhangsan', ' 23'], ['sb', ' 49'], ['wm', ' 90']]

In [14]:
rdd =rdd.map(lambda x: (x[0], int(x[1])))

In [15]:
rdd.collect()

[('zhangsan', 23), ('sb', 49), ('wm', 90)]

In [16]:
df = spark.createDataFrame(rdd, schema=['name', 'age'])

In [17]:
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)



In [18]:
df.show(20, False)

# False=> 如果数据长度超过20字节 不显示

+--------+---+
|name    |age|
+--------+---+
|zhangsan|23 |
|sb      |49 |
|wm      |90 |
+--------+---+



In [19]:
df.createTempView('people')

In [20]:
spark.sql(  ' select * from people where age <30 ').show()

+--------+---+
|    name|age|
+--------+---+
|zhangsan| 23|
+--------+---+



# 2

In [21]:
rdd = sc.textFile('people.txt').map(lambda x: x.split(',')).map(lambda x: (x[0], int(x[1])))

In [22]:
schema = StructType().add('name', StringType(), nullable=True).add('age', IntegerType(), nullable=False)

In [23]:
df = spark.createDataFrame(rdd, schema=schema)

In [24]:
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- age: integer (nullable = false)



In [25]:
df.show()

+--------+---+
|    name|age|
+--------+---+
|zhangsan| 23|
|      sb| 49|
|      wm| 90|
+--------+---+



# 3

In [26]:
df1 = rdd.toDF(['name', 'age'])

In [27]:
df1.printSchema()

root
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)



In [28]:
type(df1)

pyspark.sql.dataframe.DataFrame

# Pandas to 

In [29]:
pdf = pd.DataFrame(
    {
        
        'id': [1,2,3],
        'name': ['sb', 'dasha', 'nt'],
        'age': [10,30,50]
    }
)

In [30]:
df = spark.createDataFrame(pdf)


In [31]:
df.printSchema()

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)



# word count


# SQL

In [32]:
rdd = sc.textFile('word.txt').flatMap( lambda x: x.split(' '))  .\
    map(lambda x: [x])

In [33]:
df =rdd.toDF(['word'])

In [34]:
df.createTempView('words2')

In [35]:
spark.sql("""

    SELECT word , COUNT(*)  AS cnt 
    FROM words2 
    GROUP BY word 
    ORDER BY cnt DESC
""").show()

+------+---+
|  word|cnt|
+------+---+
|hadoop|  6|
| spark|  3|
| flink|  1|
+------+---+



# DSL

In [36]:
from pyspark.sql import functions as F

In [37]:
df = spark.read.format("text").load('word.txt')

In [38]:
df2 =df.withColumn("value", F.explode (F.split(df['value'] , ' ')))

In [39]:
df2.show()

+------+
| value|
+------+
|hadoop|
|hadoop|
| spark|
|hadoop|
|hadoop|
| spark|
|hadoop|
|hadoop|
| spark|
| flink|
+------+



In [40]:
df2.groupBy('value').count() .\
    withColumnRenamed('value', 'word').\
    withColumnRenamed('count', 'cnt').\
    orderBy('cnt', ascending = False).   show()

+------+---+
|  word|cnt|
+------+---+
|hadoop|  6|
| spark|  3|
| flink|  1|
+------+---+



# UDF

In [42]:
rdd = sc.parallelize([1,2,3,4,5,6,7,8,9]).map(lambda x:[x])

In [43]:
df = rdd.toDF(['num'])

In [51]:
def num_rid_10(x): return x*10

In [52]:
udf2 = spark.udf.register('udf1', num_rid_10, IntegerType())

# SQL

In [53]:
df.selectExpr('udf1(num)').show()

+---------+
|udf1(num)|
+---------+
|       10|
|       20|
|       30|
|       40|
|       50|
|       60|
|       70|
|       80|
|       90|
+---------+



# DSL

In [54]:
df.select(udf2(df['num'])).show()

+---------+
|udf1(num)|
+---------+
|       10|
|       20|
|       30|
|       40|
|       50|
|       60|
|       70|
|       80|
|       90|
+---------+



In [ ]:
['hadoop hadoop spark', 'hadoop hadoop spark', 'hadoop hadoop spark flink']

In [56]:
rdd = sc.parallelize([   ['hadoop hadoop spark'] ,  ['hadoop hadoop spark flink']    ])

In [57]:
df = rdd.toDF(['line'])

In [58]:
def split_line(data): return data.split(' ')

In [60]:
udf2 =  spark.udf.register('udf1', split_line, ArrayType(StringType()))

# DSL

In [61]:
df.select(udf2(df['line'])).show()

+--------------------+
|          udf1(line)|
+--------------------+
|[hadoop, hadoop, ...|
|[hadoop, hadoop, ...|
+--------------------+



# SQL

In [62]:
df.createTempView('lines')

In [65]:
spark.sql(' select udf1(line) from lines').show(truncate=False)

+------------------------------+
|udf1(line)                    |
+------------------------------+
|[hadoop, hadoop, spark]       |
|[hadoop, hadoop, spark, flink]|
+------------------------------+



# F => only DSL

In [66]:
udf3 = F.udf(split_line, ArrayType(StringType()))

In [67]:
df.select(  udf3(df['line'])  ).show(truncate=False)

+------------------------------+
|split_line(line)              |
+------------------------------+
|[hadoop, hadoop, spark]       |
|[hadoop, hadoop, spark, flink]|
+------------------------------+

